In [55]:
import os
import json
import ast
import networkx as nx
import matplotlib.pyplot as plt


In [56]:

def parse_python_folder(folder_path):
    folder_contents = {}
    
    # Iterate through all files in the directory
    for filename in os.listdir(folder_path):
        if filename.endswith(".py"):  # Process only Python files
            file_path = os.path.join(folder_path, filename)
            with open(file_path, "r", encoding="utf-8") as file:
                folder_contents[filename] = f"""{file.read()}"""
    
    return folder_contents

# Example usage
folder_path = "files"  # Change this to your actual folder path
folder_contents = parse_python_folder(folder_path)

# Convert to JSON string for output
json_output = json.dumps(folder_contents, indent=4)
folder_contents


{'a.py': 'def a():\n    a = 1\n    return a',
 'b.py': 'def b():\n    b = 3\n    return b',
 'c.py': 'def c(m,n):\n    k = m + n\n    return k',
 'd.py': 'def d():\n    d = 5\n    return d',
 'main.py': 'import a\nimport b\nimport c as calc\nimport d\n\ntemp = calc.c(b.b(),d.d())\nanswer = a.a()+temp\nprint(answer) '}

In [59]:
import ast

def extract_dependencies(node, dependencies, current_target=None, function_name=None):
    """Recursively extract dependencies from AST nodes."""
    if isinstance(node, ast.FunctionDef):
        function_name = node.name
        dependencies.setdefault(function_name, [])  
        for stmt in node.body:
            extract_dependencies(stmt, dependencies, function_name=function_name)

    elif isinstance(node, ast.Assign):
        for target in node.targets:
            if isinstance(target, ast.Name):
                dependencies.setdefault(target.id, []) 
                extract_dependencies(node.value, dependencies, target.id)

    elif isinstance(node, ast.BinOp):
        extract_dependencies(node.left, dependencies, current_target)
        extract_dependencies(node.right, dependencies, current_target)

    elif isinstance(node, ast.Call):
        func_name = None
        if isinstance(node.func, ast.Name):  
            func_name = node.func.id
        elif isinstance(node.func, ast.Attribute) and isinstance(node.func.value, ast.Name):
            func_name = node.func.value.id

        if func_name:
            dependencies.setdefault(func_name, []) 
            if current_target:
                dependencies.setdefault(current_target, []) 
                dependencies[current_target].append(func_name)  

            for arg in node.args:
                extract_dependencies(arg, dependencies, func_name) 

    elif isinstance(node, ast.Name):
        if current_target:
            dependencies.setdefault(current_target, [])  
            dependencies[current_target].append(node.id)

    elif isinstance(node, ast.Return) and function_name:
        if isinstance(node.value, ast.Name):
            dependencies.setdefault(node.value.id, []) 
            dependencies.setdefault(function_name, []) 
            dependencies[function_name].append(node.value.id)  

def parse_code(code):
    """Parse Python code and extract dependencies."""
    tree = ast.parse(code)
    dependencies = {}
    for stmt in tree.body:
        extract_dependencies(stmt, dependencies)
    
    edges = [(src, dst) for dst, sources in dependencies.items() for src in sources]
    return edges


for key in folder_contents:
    edges = parse_code(folder_contents[key])
    print(f"{key}: {edges}")

    # # Create a directed graph
    # G = nx.DiGraph()
    # G.add_edges_from(edges)
    
    # # Draw graph
    # plt.figure(figsize=(6, 6))
    # nx.draw(G, with_labels=True, node_color="lightblue", edge_color="gray", node_size=3000, font_size=12)
    # plt.show()


a.py: [('a', 'a')]
b.py: [('b', 'b')]
c.py: [('k', 'c'), ('m', 'k'), ('n', 'k')]
d.py: [('d', 'd')]
main.py: [('calc', 'temp'), ('b', 'calc'), ('d', 'calc'), ('a', 'answer'), ('temp', 'answer')]
